In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from collections import Counter
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer
from preprocessing import *

In [17]:
glove_twitter_25 = []

In [2]:
print("loading pre trained embeddings, this can take some minutes...")
glove_twitter_25 = KeyedVectors.load_word2vec_format('glove-twitter-25.txt', binary=False)
print("loading complete.")

loading pre trained embeddings, this can take some minutes...
loading complete.


In [2]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t", encoding = "utf-8")
print(dataset.shape)
dataset.dropna(inplace = True)
print(dataset.shape)
dataset.drop(dataset[dataset.rating == 3].index, inplace=True)
print(dataset.shape)
dataset.drop_duplicates(inplace = True)
print(dataset.shape)

(3150, 5)
(3150, 5)
(2998, 5)
(2322, 5)


In [3]:
X = np.array(dataset["verified_reviews"].values).reshape(-1, 1)
y = list(dataset["feedback"].values)

In [4]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state = 0)

X, y = undersampler.fit_resample(X, y)


print('Resampled dataset shape %s' % Counter(y))

Resampled dataset shape Counter({1: 442, 0: 221})


In [5]:
X_temp = []

for rev in X:
  X_temp.append(rev[0])

In [6]:
new_text, new_sent_tok = tokenize_list_of_text(X_temp, custom_stopwords, False, pos_list)

total number of types extracted is: 1810


In [7]:
def frequency_cleaning(new_sent_tok, n):
    
    tot_tokens = []

    for sent in new_sent_tok:
        for tok in sent:
            tot_tokens.append(tok)

    freqs = nltk.FreqDist(tot_tokens)
    cleaned_reviews = []

    for sent in new_sent_tok:
        clean_sent = []
        for tok in sent:
            if freqs[tok] > n:
                clean_sent.append(tok)
        cleaned_reviews.append(clean_sent)

    return cleaned_reviews

In [8]:
cleaned_reviews = frequency_cleaning(new_sent_tok, 2)

In [9]:
bigrams = Phrases(cleaned_reviews, scoring="npmi", threshold=0.60) #estrae le collocazioni tramite PMI
bigrams[cleaned_reviews][0]

['like',
 'fact',
 'answer',
 "n't_see",
 'real',
 'need',
 'household',
 'though',
 'good',
 'day',
 'deal']

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(bigrams[cleaned_reviews], y, test_size=0.20, random_state=10)

In [11]:
w2v_model = Word2Vec(X_train, vector_size=100, window = 10, min_count = 0, sg=1, hs = 1, seed = 5, epochs=100)

In [12]:
w2v_model.wv.most_similar("love", topn = 10)

[('recipe', 0.48422297835350037),
 ('great', 0.4830770790576935),
 ('cook', 0.4746050536632538),
 ('like', 0.3906935155391693),
 ('live', 0.38091951608657837),
 ('morning', 0.3785885274410248),
 ('alarm_clock', 0.3778526782989502),
 ('kitchen', 0.3760863244533539),
 ('base', 0.3586503565311432),
 ('spanish', 0.3468755781650543)]

In [13]:
w2v_model.wv.most_similar("love", topn = 2000)[-10:]

[('step', -0.0566699355840683),
 ('offer', -0.06057509407401085),
 ('seem', -0.07512812316417694),
 ('result', -0.07515031844377518),
 ('fairly', -0.07775980234146118),
 ('high', -0.07840931415557861),
 ('phone', -0.0816013291478157),
 ('time', -0.09640014916658401),
 ('standard', -0.12020003795623779),
 ('try', -0.12049003690481186)]

In [14]:
def review_vectors(tokens, size = 300, pretrained = False, pretrained_embeddings = None):
    
    """Genera un vettore per ogni recensione: questo vettore
    è calcolato come la media ponderata (t * w: token vettore * peso tfidf) dei vettori dei token nella recensione.
    Il vettore risultante è normalizzato alla fine."""

    vec = np.zeros(size).reshape((1, size))
    count = 0

    for word in tokens:
        try:
            if pretrained:
                vec += pretrained_embeddings[word]
                count +=1
            else:
                vec += w2v_model.wv[word]
                count +=1
        except KeyError:
            # print("non trovo", word)
            continue

    if count!= 0:
        vec = vec / norm(vec)
        
    return vec

In [15]:
w2v_model.vector_size

100

In [18]:
w2v_X_train = np.zeros((len(X_train), w2v_model.vector_size))
for i in range(len(X_train)):
    w2v_X_train[i,:] = review_vectors(X_train[i], w2v_model.vector_size,False, glove_twitter_25)
w2v_X_train.shape

(530, 100)

In [19]:
w2v_X_test = np.zeros((len(X_test), w2v_model.vector_size))
for i in range(len(X_test)):
    w2v_X_test[i,:] = review_vectors(X_test[i], w2v_model.vector_size,False, glove_twitter_25)
w2v_X_test.shape

(133, 100)

In [20]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks

# # Create the SMOTE object
# sm = SMOTE(sampling_strategy=1, k_neighbors=5, random_state=10)

# # Apply SMOTE to the training data
# X_train_resampled, y_train_resampled = sm.fit_resample(w2v_X_train, Y_train)

# print('Resampled dataset shape %s' % Counter(y_train_resampled))

undersampler_nn = TomekLinks(sampling_strategy="all") # use tomeLinks to remove ambiguous data through nearest neighbours


# Fit and transform the X and y data
X_train_resampled, y_train_resampled = undersampler_nn.fit_resample(w2v_X_train, Y_train)


print('Resampled dataset shape %s' % Counter(y_train_resampled))

Resampled dataset shape Counter({1: 330, 0: 158})


In [21]:
from keras.layers import Flatten

In [22]:
from keras.utils import to_categorical
Y_train_hot = to_categorical(y_train_resampled)

In [23]:
Y_test_hot = to_categorical(Y_test)

In [25]:
from keras.callbacks import EarlyStopping

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='binary_accuracy', patience=25)

# Fit the model with early stopping
model = Sequential()
model.add(Dense(300, activation='relu', input_dim=w2v_model.vector_size))
model.add(Dropout(0.2))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

# Compile your model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
model.fit(X_train_resampled, Y_train_hot, epochs=100, batch_size=12, verbose=2, validation_split=0.1, callbacks = [early_stopping])

Epoch 1/100
37/37 - 1s - loss: 0.5939 - binary_accuracy: 0.6720 - val_loss: 0.4673 - val_binary_accuracy: 0.6939 - 1s/epoch - 37ms/step
Epoch 2/100
37/37 - 0s - loss: 0.3996 - binary_accuracy: 0.8178 - val_loss: 0.3164 - val_binary_accuracy: 0.8163 - 131ms/epoch - 4ms/step
Epoch 3/100
37/37 - 0s - loss: 0.2733 - binary_accuracy: 0.8827 - val_loss: 0.3229 - val_binary_accuracy: 0.8265 - 192ms/epoch - 5ms/step
Epoch 4/100
37/37 - 0s - loss: 0.2201 - binary_accuracy: 0.8941 - val_loss: 0.3505 - val_binary_accuracy: 0.8367 - 154ms/epoch - 4ms/step
Epoch 5/100
37/37 - 0s - loss: 0.1800 - binary_accuracy: 0.9203 - val_loss: 0.3679 - val_binary_accuracy: 0.8367 - 154ms/epoch - 4ms/step
Epoch 6/100
37/37 - 0s - loss: 0.1570 - binary_accuracy: 0.9260 - val_loss: 0.3857 - val_binary_accuracy: 0.8571 - 139ms/epoch - 4ms/step
Epoch 7/100
37/37 - 0s - loss: 0.1037 - binary_accuracy: 0.9419 - val_loss: 0.4239 - val_binary_accuracy: 0.8367 - 130ms/epoch - 4ms/step
Epoch 8/100
37/37 - 0s - loss: 0.063

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 300)               30300     
                                                                 
 dropout_3 (Dropout)         (None, 300)               0         
                                                                 
 dense_5 (Dense)             (None, 300)               90300     
                                                                 
 dropout_4 (Dropout)         (None, 300)               0         
                                                                 
 dense_6 (Dense)             (None, 100)               30100     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 2)                

In [27]:
predictions = model.predict(w2v_X_test) 

5/5 [==============================] - 0s 2ms/step


In [28]:
predictions = np.round(predictions)

In [29]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_hot, predictions))

              precision    recall  f1-score   support

           0       0.70      0.90      0.79        42
           1       0.95      0.82      0.88        91

   micro avg       0.85      0.85      0.85       133
   macro avg       0.83      0.86      0.84       133
weighted avg       0.87      0.85      0.85       133
 samples avg       0.85      0.85      0.85       133

